#### Importing Required Modules 

In [147]:
# Importing BeautifulSoup for parsing HTML document from webpage
from bs4 import BeautifulSoup
from tqdm import tqdm

# Importing selenium and Chromedriver_binary
from selenium import webdriver
import chromedriver_binary

# Importing Data Processing and Lninear Algebra Module
import pandas as pd
import numpy as np 

import time

In [148]:
# Initiating the chrome driver in the system ,which will will pop up as an individual chrome testing browser as
driver = webdriver.Chrome()

In [149]:
# Providing the link
link = 'https://stock-pictures.netlify.app/'
driver.get(link)

#### Auto window scroll to reach  bottom

In [150]:
# Auto Window Scroll to reach bottom of the webpage
for i in tqdm(range(0,33000000,1000)):
    driver.execute_script("window.scrollTo(0,"  + str(i) + ")")
    time.sleep(.1)

100%|███████████████████████████████████████████████████████████████████| 3300/3300 [06:01<00:00,  9.13it/s]


#### Data Scraping

In [173]:
#Parsing the HTML code
soup = BeautifulSoup(driver.page_source,'html.parser')

#Checking number of images to be scraped
len(soup.find_all('div',class_ = 'container'))

853

In [160]:
#Scraping the Image Details

data = [] # We will be storing all the data as a list 

for sp in soup.find_all('div',class_ = 'container'):
    #print(sp.find('img'))
    if(('.jpg'or'.png') in sp.find('img').get('src')):
        link = sp.find('img').get('src')
        tag = list(set(sp.find('div',class_ = 'tags').text[7:].split(" ")))
        like = int(sp.find('div',class_ = 'likes-comments').find_all('span')[0].text[:-6])
        comment = int(sp.find('div',class_ = 'likes-comments').find_all('span')[1].text[:-9])
        #print(link,' '.join(tag),like,comment)
        
         #Appending new scraped data into current data list with old data
        data.append([link,' '.join(tag),like,comment])

In [188]:
len(data)

435

#### Saving data in dataframe

In [189]:
df = pd.DataFrame(data,columns=['link','tags','likes','comments'])  # Creating dataframe from the data list 
df.head()

,link,tags,likes,comments
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Clouds, Sky Blue Atmosphere, Sky,",196,55
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Ornithology, Hummingbird",76,20
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Sea, Subtropical Rainfall, Rainbow,",282,106
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"Sakura Road, Blossoms, Cherry Japan,",42,11
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"Plant Cape Marguerite, Flower,",39,15


#### Downloading the image and saving in 'image_folder' @ current directory

In [185]:
paths = []
img_ids = []
ref = 100000
i = 1

for link in tqdm(df['link']): # tqdm visualize the iteration
    res = requests.get(link)  # sending request
    
    img_id = 'A' + str(i+ref)[1:] # creating image ID
    i = i+1
    img_path = 'image_folder/'+ img_id + '.jpg' # Do create a new folder name " image_folder " in current working directory
    
    # File Descriptor
    fd = open(img_path,'wb') # wb means Web Binary
    fd.write(res.content) # saving image in web binary format
    fd.close()
    
    paths.append(img_path) # Saving the path of every downloaded image 
    img_ids.append(img_id) # Saving ID/Name given to every images
    
    

100%|█████████████████████████████████████████████████████████████████████| 435/435 [01:00<00:00,  7.15it/s]


In [191]:
df['img_path'] = paths    # storing the path list in a new coloumn 'img_paths' 
df['img_ids'] = img_ids   # storing the path list in a new coloumn 'img_ids' 

#### Saving final dataset as csv file

In [163]:
df.to_csv('stock_photo.csv',index=False) # Exporting the data in to CSV file named "data.csv"

In [165]:
df = pd.read_csv('stock_photo.csv')  # Importing that same data.csv into notebook
df

,link,tags,likes,comments,path
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Clouds, Sky Blue Atmosphere, Sky,",196,55,image_folder/clouds-7050884__480.jpg
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Ornithology, Hummingbird",76,20,image_folder/bird-7117346__340.jpg
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Sea, Subtropical Rainfall, Rainbow,",282,106,image_folder/sea-7039471__340.jpg
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"Sakura Road, Blossoms, Cherry Japan,",42,11,image_folder/cherry-blossoms-7110279__340.jpg
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"Plant Cape Marguerite, Flower,",39,15,image_folder/cape-marguerite-7121992__340.jpg
...,...,...,...,...,...
430,https://cdn.pixabay.com/photo/2022/02/21/21/18...,"Animal, Rodent, Baby Daisy Mouse,",47,18,image_folder/animal-7027636__340.jpg
431,https://cdn.pixabay.com/photo/2021/10/05/16/53...,"Shrine, Costume, Prayer, Worship Kimono,",62,9,image_folder/kimono-6683245__340.jpg
432,https://cdn.pixabay.com/photo/2022/01/10/15/30...,"Castle, Star, Historical Fantasy, Night,",111,87,image_folder/castle-6928593__340.jpg
433,https://cdn.pixabay.com/photo/2022/03/31/09/50...,"Blue, Abstract, Bright Arch, Background,",7,0,image_folder/abstract-7102657__480.jpg


In [168]:
df['link'].describe()

count                                                   435
unique                                                  434
top       https://cdn.pixabay.com/photo/2022/04/10/19/28...
freq                                                      2
Name: link, dtype: object

In [175]:
# Checking null values in dataframe
df.isnull().sum()

link        0
tags        0
likes       0
comments    0
path        0
dtype: int64

In [171]:
# Removing uplicates in dataframe
df = df.drop_duplicates()